In [1]:
import pandas as pd
import numpy  as np

from pygeodyn.pygeodyn_plot_scalingfactors import *
import os


# Proccess data


In [2]:
# month_list = ['oct', 'nov', 'dec']
month_list = ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']#,

run_list = ['msis2',
            'jb2008',
            'dtm2020_o',
            'hasdm_oc',
           ]
    
gfo_acc_file = 'gfo_acc_6month.csv'
gfo_pod_file = 'gfo_pod_6month.csv'
ice_pod_file = 'icesat2_6month.csv'
scale_cadence=3

## GFO Acc

In [3]:
if os.path.exists(gfo_acc_file) :
    print(gfo_acc_file, ' exists' )

elif os.path.exists(gfo_acc_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ gfo_acc_file+'.bz2')
# gfo_6month.csv.bz2
else:
    gfo_bigdf = {}

    for imonth,month in enumerate(month_list):
        if month=='oct':
            m_num = 10
            y_num = 2018
        if month=='nov':
            m_num = 11
            y_num = 2018
        if month=='dec':
            m_num = 12
            y_num = 2018
        if month=='jan':
            m_num = 1
            y_num = 2019
        if month=='feb':
            m_num = 2
            y_num = 2019
        if month=='mar':
            m_num = 3
            y_num = 2019
        if month=='apr':
            m_num = 4
            y_num = 2019

        path_gfo     = "/data/SatDragModelValidation/data/inputs/raw_inputdata/data_GRACEFO_accelerometer/"
        filename_gfo = f'GC_DNS_ACC_{y_num}_{m_num:02d}_v02c.txt'

        datapath_gfo     = path_gfo + filename_gfo

        headers = [
            'date',        #         Date (yyyy-mm-dd)
            'time',        #         Time (hh:mm:ss.sss)
            'time_system', #         Time system: UTC or GPS (differs per mission)
            'alt',         #  f10.3  Altitude (m), GRS80
            'lon',         #   f8.3  Geodetic longitude (deg), GRS80
            'lat',         #   f7.3  Geodetic latitude (deg), GRS80
            'lst',         #   f6.3  Local solar time (h)
            'arglat',      #   f7.3  Argument of latitude (deg)
            'dens_x',      #  e15.8  Density derived from accelerometer measurements (kg/m3)
            'dens_mean',   #  e15.8  Running orbit average of density (kg/m3)
            'flag_den',    #   f4.1  Flag for density: 0 = nominal data, 1 = anomalous data (-)
            'flag_orbitavg',#   f4.1  Flag for running orbit average density: 0 = nominal data, 1 = anomalous data (-)
                    ]

        print(month)
        gfo_bigdf[month] = pd.read_csv(datapath_gfo, 
                skiprows = 38, 
                sep = '\s+',
                names = headers,
                           )

        #Convert date from GPS to UTC
        date = pd.to_datetime(gfo_bigdf[month]['date']  \
                            + gfo_bigdf[month]['time'], \
                                    format='%Y-%m-%d%H:%M:%S.000') - pd.to_timedelta(18,'s')

        gfo_bigdf[month].insert(0, 'Date', date)

        del gfo_bigdf[month]['date'], gfo_bigdf[month]['time'], date
        del gfo_bigdf[month]['time_system']
        del gfo_bigdf[month]['dens_mean']
        del gfo_bigdf[month]['flag_den']
        del gfo_bigdf[month]['flag_orbitavg']


    #     resid_meas_summry = pd.concat([ gfo_bigdf, resid_meas_summry_iter])

    gfo_acc_df = pd.concat([ gfo_bigdf[month] for month in month_list]  )
    gfo_acc_df = gfo_acc_df.reset_index(drop=True)

    

if os.path.exists(gfo_acc_file) :
    print(gfo_acc_file, ' exists' )
    
    
    gfo_acc_df = pd.read_csv(gfo_acc_file, 
                    sep = ',',
                    )

    os.system('bzip2 -v '+ gfo_acc_file)

    
else:
    print(f"---Calculating Grace-FO normalization")
    D500_gfo = normalize_density_msis2( gfo_acc_df , 'GRACE-FO Acc', 500)
    gfo_acc_df['D500_gfo'] = D500_gfo
    #### save to a csv
    gfo_acc_df.to_csv(gfo_acc_file, index=False)  

    

# gfo_df2 =     gfo_df.query("Date >= '2018-10-14' and Date < '2018-12-30'")
# del gfo_df
    

<>:58: SyntaxWarning:

invalid escape sequence '\s'

<>:58: SyntaxWarning:

invalid escape sequence '\s'

/tmp/ipykernel_3657/2296476006.py:58: SyntaxWarning:

invalid escape sequence '\s'

  gfo_acc_6month.csv.bz2: 

must bunzip


done


gfo_acc_6month.csv  exists


  gfo_acc_6month.csv:  3.287:1,  2.434 bits/byte, 69.58% saved, 176047371 in, 53552927 out.


## GFO POD

In [4]:
if os.path.exists(gfo_pod_file):
    print(gfo_pod_file, ' exists' )

    
elif os.path.exists(gfo_pod_file+'.bz2') :
    
    print('must bunzip')
    os.system('bunzip2 -v '+ gfo_pod_file+'.bz2')
    
else:


    %load_ext autoreload
    %autoreload 2
    from gc import collect as gc_collect
    import pickle 
    from datetime import datetime,timedelta


#     month_list = ['nov']


    scale_cadence = 3

    dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
    run_dict={}
    for imonth,month in enumerate(month_list):
        for i in run_list:
            if i =='msis2':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 5
                run_dict[month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 3
                run_dict[month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 1
                run_dict[month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[month+i]={}
                run_dict[month+i]['num'] = 2 
                run_dict[month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv

    print(run_dict)



    dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
                 + 'gracefoc/cross_validation/dria_3hrscale/'
    obj = {}
    for imonth,month in enumerate(month_list):
        for i,model in enumerate(run_list):
            pickleName = f'_{month}_DRIA_scale{scale_cadence}.pkl'

            ### Load the data if the pickles exist
            print()
            print()
            gc_collect()

            pickle_file = dir_save+model+pickleName
            
            obj[month+model] = pd.read_pickle(pickle_file)

            print('Loaded data from pickle... ',  month+model)


    ### Save space if doing density retrieval
    for model in run_dict.keys():
        del obj[model]['OrbitResids']
        del obj[model]['Trajectory_orbfil']

    gc_collect()
    
    
    ## MAKE SCALING FACTORS
    satid = 2012003
    wgts = {}

    for model in run_dict.keys():
        wgts[model] = {}
        ScalingFactors  = []
        ScalingFactor_times = []

        for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
            epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj[model]['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
            ### Collect the weights for the ensemble average
            inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
            wgts[model][arc_name] = inv_rms#/sum_wgts

            iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
            for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
                if iit == 0 or iit==9:
                    pass
                else:
                    CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                    APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                    ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                    ScalingFactor_times.append(itime)
        run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
        run_dict[model]['ScalingFactors']      = ScalingFactors
        run_dict[model]['Weight'] = wgts[model]



    ###### Scale the densities
    models_dens = {}
    for monthmodel in run_dict.keys():
        print(f"---Making continuous scaled rho for {monthmodel}")
        models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

        del obj[monthmodel]
        
    gc_collect()               
    ## Retrieve scaled ensemble weighted average
    ##     'Rho_x' denotes the ensemble weighted avg
    print(f"---Making ensemble avg")
#     for monthmodel in run_dict.keys():
#         month = monthmodel[:3]
    models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, month_list, run_list)


    ### Clear up space 
    del obj
    for monthmodel in run_dict.keys():
        print(monthmodel)
        if 'Rho_x' not in monthmodel:
            del models_dens[monthmodel]

    gc_collect()       
    
    
    
if os.path.exists(gfo_pod_file) :
    print(gfo_pod_file, ' exists' )
    
    gfo_pod_df = pd.read_csv(gfo_pod_file, 
                    sep = ',',
                    )

    os.system('bzip2 -v '+ gfo_pod_file)

else:
    
    gfo_pod_bigdf = {}
    print(f"---Calculating gfo_pod normalization")
    
    for imonth,month in enumerate(month_list):
        print(month)
        
        D500_gfo_pod = normalize_density_msis2( models_dens[month+'Rho_x'], 'GFO-C POD', 500)
        models_dens[month+'Rho_x']['D500_gfo_pod'] = D500_gfo_pod
        
        gfo_pod_bigdf[month] = pd.DataFrame.from_dict(models_dens[month+'Rho_x'])
        
        del models_dens[month+'Rho_x']

    print('--concatenating and saving')

    gfo_pod_df = pd.concat([ gfo_pod_bigdf[month] for month in month_list]  )
    gfo_pod_df = gfo_pod_df.reset_index(drop=True)
    gfo_pod_df.to_csv(gfo_pod_file, index=False)  

    ### Need to set this up to only write to file once. and have that write include the full dataset. Maybe concat the dataframes by month first



gfo_pod_6month.csv  exists
gfo_pod_6month.csv  exists


In [5]:
gfo_pod_df

,date,lon,lat,alt,Rho_x,Rho_std,D500_gfo_pod
0,2018-10-23 00:00:09,8.96520,-57.192100,530478.173926,2.112706e-14,4.560842e-15,3.869462e-14
1,2018-10-23 00:00:19,8.96120,-57.820900,530705.461654,2.121141e-14,4.585969e-15,3.903554e-14
2,2018-10-23 00:00:29,8.95850,-58.449500,530928.402131,2.130455e-14,4.612768e-15,3.939189e-14
3,2018-10-23 00:00:39,8.95720,-59.078100,531146.903858,2.140460e-14,4.638666e-15,3.976024e-14
4,2018-10-23 00:00:49,8.95740,-59.706500,531360.876336,2.151063e-14,4.662355e-15,4.013878e-14
...,...,...,...,...,...,...,...
1494715,2019-04-30 23:59:19,333.78090,-51.118799,517343.742894,2.765991e-14,3.857862e-15,3.604627e-14
1494716,2019-04-30 23:59:29,333.76635,-50.486999,516949.296140,2.777942e-14,3.908356e-15,3.605300e-14
1494717,2019-04-30 23:59:39,333.75110,-49.855024,516552.915138,2.790402e-14,3.964432e-15,3.606142e-14
1494718,2019-04-30 23:59:49,333.73520,-49.222974,516154.721386,2.803388e-14,4.025961e-15,3.607170e-14


In [6]:
import linecache
import pandas as pd
import numpy as np

path_to_flux = "/data/SatDragModelValidation/src/geodyn_code/support/geodyn_support/make_tables/"


dateS   = []
dateAP  = []
dateKP8 = []

fluxS   = []
fluxAP  = []
fluxKP8 = []

switch = 0
with open(path_to_flux+"new_master", 'r') as f:
    for line_no, string in enumerate(f):
#         string = linecache.getline("new_master",line+1) #

        if 'MASTER' in string:        
            switch = 'skip'
            continue
            
        ### Skip the title lines and any dates before Y2K
        if 'FLUXS' in string:        
            switch = 'readfluxS'
            print(switch)
            continue

        if 'FLUXAP' in string:        
            switch = 'readfluxAP'
            print(switch)
            continue
        if 'FLUXKP8' in string:        
            switch = 'readfluxKP8'
            print(switch)
            continue

        if switch =='skip':
            continue
            
        if int(string[:2]) >= 58:
            continue
        else:
            
            if int(string[:2]) < 18 or int(string[:2]) > 19:
                continue
                
            if switch == 'readfluxS':
                ### Load the Solar Flux values after 2000. The date leading each line represents
                ### the first flux value in that line.  The following n values are that day + n days.
                ###  All Solar Flux values are mulitplied by 10 for some reason...

                for n in range(0,12):
                    dateS.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxS.append(int(string[11:15]) / 10)
                fluxS.append(int(string[16:20]) / 10)
                fluxS.append(int(string[21:25]) / 10)
                fluxS.append(int(string[26:30]) / 10)
                fluxS.append(int(string[31:35]) / 10)
                fluxS.append(int(string[36:40]) / 10)
                fluxS.append(int(string[41:45]) / 10)
                fluxS.append(int(string[46:50]) / 10)
                fluxS.append(int(string[51:55]) / 10)
                fluxS.append(int(string[56:60]) / 10)
                fluxS.append(int(string[61:65]) / 10)
                fluxS.append(int(string[66:70]) / 10)

            if switch == 'readfluxAP':
                ### Load the AP values after 2000. These are formatted the same as the FLUXS section
                for n in range(0,12):
                    dateAP.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxAP.append(int(string[11:15]) / 10)
                fluxAP.append(int(string[16:20]) / 10)
                fluxAP.append(int(string[21:25]) / 10)
                fluxAP.append(int(string[26:30]) / 10)
                fluxAP.append(int(string[31:35]) / 10)
                fluxAP.append(int(string[36:40]) / 10)
                fluxAP.append(int(string[41:45]) / 10)
                fluxAP.append(int(string[46:50]) / 10)
                fluxAP.append(int(string[51:55]) / 10)
                fluxAP.append(int(string[56:60]) / 10)
                fluxAP.append(int(string[61:65]) / 10)
                fluxAP.append(int(string[66:70]) / 10)

            if switch == 'readfluxKP8':
                ### Load the KP values after 2000. These are formatted such that each line is a day
                ### and each nth value in the line is n*3hrs for the day
                for n in range(0,8):
                    dateKP8.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(3,'h') )
                fluxKP8.append(int(string[7:11])  / 100)
                fluxKP8.append(int(string[12:16]) / 100)
                fluxKP8.append(int(string[17:21]) / 100)
                fluxKP8.append(int(string[22:26]) / 100)
                fluxKP8.append(int(string[27:31]) / 100)
                fluxKP8.append(int(string[32:36]) / 100)
                fluxKP8.append(int(string[37:41]) / 100)
                fluxKP8.append(int(string[42:46]) / 100)

fluxS = np.array(fluxS)
fluxAP = np.array(fluxAP)
fluxKP8 = np.array(fluxKP8)
                
fluxS   = np.where(fluxS  == 0., np.nan, fluxS  )
fluxAP  = np.where(fluxAP ==0., np.nan, fluxAP )
fluxKP8 = np.where(fluxKP8==0., np.nan, fluxKP8)



readfluxS
readfluxAP
readfluxKP8


In [7]:
# gfo_acc_df['dens_x']

# Filter for Commonalities

In [8]:
gfo_acc_df = gfo_acc_df.query("Date >= '2018-10-15' and Date < '2019-05-01'")
gfo_pod_df = gfo_pod_df.query("date >= '2018-10-15' and date < '2019-05-01'")
# ice_pod_df = ice_pod_df.query("date >= '2018-10-15' and date < '2019-05-01'")

(timeavg_gfo_acc,  denavg_gfo_acc) = orbit_avg_generic(gfo_acc_df['Date'], gfo_acc_df['dens_x'], gfo_acc_df['lat'])    
(timeavg_gfo_pod,  denavg_gfo_pod) = orbit_avg_generic(gfo_pod_df['date'], gfo_pod_df['Rho_x'], gfo_pod_df['lat'])    
# (timeavg_gfo_acc,  denavg_gfo_acc) = orbit_avg_generic(gfo_acc_df['Date'], gfo_acc_df['D500_gfo'], gfo_acc_df['lat'])    
# (timeavg_gfo_pod,  denavg_gfo_pod) = orbit_avg_generic(gfo_pod_df['date'], gfo_pod_df['D500_gfo_pod'], gfo_pod_df['lat'])    



In [20]:
def calc_m_to_o_logspace(model, obs):
    import numpy as np
#     return(  np.exp(np.log(model/obs)))
    return( np.log(model/obs))


In [53]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

stat = {}
stat['time_midpoint_GFO'] = []
stat['gfo_acc_denorbavg'] = []
stat['gfo_pod_denorbavg'] = []

stat['gfo_acc_time']      = []
stat['gfo_pod_time']      = []
# 

# 
stat['ratio_GFO']    = []
stat['m_to_o_GFO']    = []


from numpy import exp as np_exp
from numpy import log as np_log # this is the natural log


for i,val in enumerate( timeavg_gfo_acc):
    
    date_near = nearest(timeavg_gfo_pod, val)
    indx = np.where(pd.to_datetime(timeavg_gfo_pod) == pd.to_datetime(date_near))[0][0]
    
    stat['gfo_acc_time'].append( val)
    stat['gfo_acc_denorbavg'].append( denavg_gfo_acc[i])

    if np.abs((val - timeavg_gfo_pod[indx]).total_seconds()) > 5640:  # only compute % change if values within  orbit (95mins)
        pass
    
    else:
        stat['time_midpoint_GFO'].append( pd.Timestamp(val) + (pd.Timestamp(timeavg_gfo_pod[indx]) - pd.Timestamp(val)) / 2)
        stat['gfo_pod_time'].append( timeavg_gfo_pod[indx])

        if np.abs((timeavg_gfo_pod[indx] - timeavg_gfo_pod[indx-1]).total_seconds()) > 10000:
            print('skipping gfo', indx, timeavg_gfo_pod[indx])
            stat['gfo_pod_denorbavg'].append( np.nan)
            stat['ratio_GFO'].append(         np.nan) 
            stat['m_to_o_GFO'].append(         np.nan) 
            
        else:
            stat['gfo_pod_denorbavg'].append( denavg_gfo_pod[indx])
            stat['ratio_GFO'].append(denavg_gfo_pod[indx]/denavg_gfo_acc[i])
            stat['m_to_o_GFO'].append(calc_m_to_o_logspace(denavg_gfo_pod[indx],denavg_gfo_acc[i])) 

        
        
############################################################################
    
N = len(stat['ratio_GFO'])
sum_log_ratio = 0
sum_stddev    = 0
sum_log_ratio_square = 0

for i in range(N): 
    # skip nans in the dataset
    if np.isnan(stat['ratio_GFO'][i]):
        continue
    else:
        sum_log_ratio        +=  np.log(stat['ratio_GFO'][i])        
        sum_log_ratio_square +=  np.log(stat['ratio_GFO'][i])**2

MU_ratio   = np.exp( (1/N)*sum_log_ratio )
RMS_ratio  = np.sqrt((1/N)*sum_log_ratio_square)  

for i in range(N): 
    # skip nans in the dataset
    if np.isnan(stat['ratio_GFO'][i]):
        continue
    else:
        sum_stddev += ( np.log(stat['ratio_GFO'][i]) - np.log(MU_ratio) )**2     
STDEV_ratio = np.sqrt((1/N)*sum_stddev)  

STDEV_ratio_percent = 100*(np.exp( STDEV_ratio )-1)
RMS_ratio_percent   = 100*(np.exp( RMS_ratio   )-1)

print(f" ")
print(f"GFO POD Stats (m/o)=(rho_pod/rho_acc) (not normalized) ")
print(f"     MU(m/o)     : {MU_ratio:9.3f}")
print(f"     STDEV(m/o)% : {STDEV_ratio_percent:9.3f} %")
print(f"     RMSe(m/o)%  : {RMS_ratio_percent:9.3f} %")



skipping gfo 0 2018-10-23 01:19:39
skipping gfo 139 2018-11-01 04:24:44
skipping gfo 139 2018-11-01 04:24:44
skipping gfo 140 2018-11-01 07:33:49
skipping gfo 596 2018-12-01 04:42:24
skipping gfo 596 2018-12-01 04:42:24
skipping gfo 597 2018-12-01 07:51:29
skipping gfo 1068 2019-01-01 04:38:14
skipping gfo 1068 2019-01-01 04:38:14
skipping gfo 1069 2019-01-01 07:47:24
skipping gfo 1614 2019-02-14 00:29:04
skipping gfo 1614 2019-02-14 00:29:04
skipping gfo 1615 2019-02-22 01:45:34
skipping gfo 1724 2019-03-01 03:58:34
skipping gfo 1724 2019-03-01 03:58:34
skipping gfo 1725 2019-03-01 07:07:44
skipping gfo 1995 2019-03-19 12:29:34
skipping gfo 1995 2019-03-19 12:29:34
skipping gfo 1996 2019-03-20 01:53:24
skipping gfo 2181 2019-04-01 03:53:14
skipping gfo 2181 2019-04-01 03:53:14
skipping gfo 2182 2019-04-01 07:02:24
 
GFO POD Stats (m/o)=(rho_pod/rho_acc) (not normalized) 
     MU(m/o)     :     1.024
     STDEV(m/o)% :     7.952 %
     RMSe(m/o)%  :     8.330 %


In [52]:
from datetime import datetime,timedelta

gfopodcolor = "#d62728"
icepodcolor = "#1f77b4"
gfoacccolor = 'black'


# fig = make_subplots(rows=1, cols=1)
fig = make_subplots(rows=3, cols=1, row_heights=[0.2, 0.4, 0.4 ],#],
                    specs=[[{"secondary_y": True}],
                           [{"secondary_y": False}],
#                            [{"secondary_y": False}],
#                            [{"secondary_y": False}],
                           [{"secondary_y": False}]],
                           shared_xaxes=True,
                           vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=dateS,
                           y=fluxS,
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
                           showlegend=False),
                           secondary_y=True,row=1, col=1)

fig.add_trace(go.Scatter(x=dateKP8,
                           y=fluxKP8,
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh', color = 'grey', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1) 



fig.add_trace(go.Scatter(x= stat['gfo_acc_time'],
                            y=stat['gfo_acc_denorbavg'],
                            name=f'GFO_acc_500km',
                            mode='lines',
                            opacity=1,
                            marker=dict( size=4, color=gfoacccolor),
                            line = dict(color = gfoacccolor, width=1),
                            showlegend=False),
                            row=2, col=1)

fig.add_trace(go.Scatter(x=stat['gfo_pod_time'],
                            y=stat['gfo_pod_denorbavg'],
                            name=f'D500_gfo_pod',
                            mode='lines',
                            opacity=1,
                            marker=dict( size=4, color=gfopodcolor),
                             line = dict(color = gfopodcolor, width=1),
                            showlegend=False),
                            row=2, col=1)


fig.add_trace(go.Scatter(x=stat['time_midpoint_GFO'],
                            y=stat['m_to_o_GFO'],

                            mode='lines',
                            opacity=1,
                            marker=dict( size=2, color=gfopodcolor),
                           line = dict(color = gfopodcolor, width=1),
                            showlegend=False),
                            row=3, col=1)






#### FANCY LEGEND ################################################################
modelnames=[]
modelcolors = []
modelnames.append("GRACE-FO Acc")
# modelnames.append(r"$\rho_{GFO,Acc}$")
modelcolors.append("black")
modelnames.append("GRACE-FO POD")
# modelnames.append(r"$\rho_{GFO,POD}$")
modelcolors.append('#d62728')
# modelnames.append("ICESat2 POD")
# modelcolors.append('#1f77b4')

df_leg = pd.DataFrame({"starts_colors": modelcolors})
fig.update_traces(showlegend=False).add_traces(
    [   go.Scattergl(name=modelnames[i],
               x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
               mode='lines',
               line = dict(shape = 'hv',  width=10),
               marker_color=c,
               showlegend=True)
        for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
## Legend Control
fig.update_layout(legend=dict(
    yanchor="top",
    y=.78,
    xanchor="center",
    x=.45,
    orientation="h",
        font=dict(family='Arial', size=12,color='black'),
        bgcolor="white",
        bordercolor="darkgrey",
        borderwidth=0.5,)  )
################################################################################

### UPDATE AXES 
fig.update_yaxes(title_text="Kp Index", 
                 exponentformat= 'power',
                 range=[0,7],
                 secondary_y=False,
                 row=1, col=1)
fig.update_yaxes(title_text="F10.7 (sfu)", 
                 exponentformat= 'power',
                 range=[20,90],
                 secondary_y=True,
                 tickfont=dict(color="blue"),
                 titlefont=dict(color="blue"),
                 row=1, col=1)
################################################################################

### SYLIZE LEGEND 
font_dict=dict(family='Arial',size=14,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = False
    else:
        if i < len(rownum):
            L_ticklabel = False
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)


fig.update_yaxes(title_text="Orbit Averaged Densities", 
                 type="log", 
                 exponentformat= 'power',row=2, col=1)

# xaxis_title=r'$\sqrt{(n_\text{c}(t|{T_\text{early}}))}$',

fig.update_yaxes(title_text=r"$\text{Ratio of Orbit Avgs.}\\ \text{    }\ln(\rho_{POD}/\rho_{Acc})$", #"Ratio of Orbit Avgs ()", 
                minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
                 exponentformat= 'power',row=3, col=1,
                                    range= [-0.5,0.5])
#                 )

# fig.update_yaxes(title_text="GFO_acc-ICE_pod %d", 
#                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=4, col=1)
# fig.update_yaxes(title_text="GFO_pod-ICE_pod %d", 
#                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=5, col=1)

fig.update_xaxes(range=[pd.to_datetime( "181020-000000", format='%y%m%d-%H%M%S'),
                        pd.to_datetime( "190503-000000", format='%y%m%d-%H%M%S')])


fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
                  autosize=False,    width=1000,    height=700,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict, plot_bgcolor='white', 
                 )

fig.show(config=config)

pio.write_image(fig, 'plot_GFOacc_v_GVOpod.jpg', scale=5)


In [12]:

# gfopodcolor = "#d62728"
# icepodcolor = "#1f77b4"
# gfoacccolor = 'black'


# # fig = make_subplots(rows=1, cols=1)
# fig = make_subplots(rows=3, cols=1, row_heights=[0.2, 0.5, 0.3 ],#],
#                     specs=[[{"secondary_y": True}],
#                            [{"secondary_y": False}],
# #                            [{"secondary_y": False}],
# #                            [{"secondary_y": False}],
#                            [{"secondary_y": False}]],
#                            shared_xaxes=True,
#                            vertical_spacing=0.02)

# fig.add_trace(go.Scatter(x=dateS,
#                            y=fluxS,
#                            name= 'F107d_1AU',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
#                            showlegend=False),
#                            secondary_y=True,row=1, col=1)

# fig.add_trace(go.Scatter(x=dateKP8,
#                            y=fluxKP8,
#                            name= 'Kp',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh', color = 'grey', width=2),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1) 



# fig.add_trace(go.Scatter(x= perc_stat['gfo_acc_time'],
#                             y=perc_stat['gfo_acc_denorbavg'],
#                             name=f'GFO_acc_500km',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color=gfoacccolor),
#                             showlegend=False),
#                             row=2, col=1)

# fig.add_trace(go.Scatter(x=perc_stat['gfo_pod_time'],
#                             y=perc_stat['gfo_pod_denorbavg'],
#                             name=f'D500_gfo_pod',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color=gfopodcolor),
#                             showlegend=False),
#                             row=2, col=1)

# # fig.add_trace(go.Scatter(x=perc_stat['ice_pod_time'],
# #                             y=perc_stat['ice_pod_denorbavg'],
# #                             name=f'D500_ice_pod',
# #                             mode='markers',
# #                             opacity=1,
# #                             marker=dict( size=4, color=icepodcolor),
# #                             showlegend=False),
# #                             row=2, col=1)



# fig.add_trace(go.Scatter(x=perc_stat['time_midpoint_GFO'],
#                             y=perc_stat['percdiff_GFO_log'],
# #                             name=f'D500_icesat2',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=3, color=gfopodcolor),
#                             showlegend=False),
#                             row=3, col=1)
# window_size = 80
# i = 0
# # Initialize an empty list to store moving averages
# moving_averages = []
# # Loop through the array to consider
# # every window of size 3
# while i < len(perc_stat['time_midpoint_GFO']) - window_size + 1:
#     # Store elements from i to i+window_size
#     # in list to get the current window
#     window = perc_stat['percdiff_GFO_log'][i : i + window_size]
#     # Calculate the average of current window
#     window_average = round(sum(window) / window_size, 2)
#     # Store the average of current
#     # window in moving average list
#     moving_averages.append(window_average)
#     # Shift window to right by one position
#     i += 1
# fig.add_trace(go.Scatter(x=perc_stat['time_midpoint_GFO'],
#                             y=moving_averages,
#                             opacity=1,
#                            line = dict( color =gfopodcolor, width=4),
#                             showlegend=False),
#                             row=3, col=1)





# N = len(perc_stat['ratio_GFO'])
# sum_log = 0
# sum_log2 = 0
# for i in range(N):
#     sum_log  += np_log(perc_stat['ratio_GFO'][i])
#     sum_log2 += np_log(perc_stat['ratio_GFO'][i])**2
# #         np_exp((1/N)*perc_stat['percdiff_log2'])-1 )*100
# mean_ratio    = np_exp( (1/N)*sum_log  )
# per_rms_diff_log = 100* (np_exp( np.sqrt((1/N)*(sum_log2))  )-1)
# print(f"mean_ratio (log):     % {mean_ratio:6.3f}")
# print(f"% RMS (log): % {per_rms_diff_log:6.3f}")
# fig.add_annotation(
#                 y= .03,
#                 x= .92,
#                 xref="paper", yref="paper",
#                 showarrow=False,
# #                 text=f"G_acc - G_pod = {mean_ratio:05.3f}% | % rms diff = {per_rms_diff_log:05.3f}%", 
#                 text=f"G_pod/G_acc = {mean_ratio:05.3f} | % RMSe = {per_rms_diff_log:05.3f}%", 
#                 align = 'left',
#                 valign = 'bottom',
#                 bgcolor='white',
#                 bordercolor=gfopodcolor,
#                 borderwidth=2.5,
#                 borderpad=2,
#                 font=dict(family='Arial',size=14,color="black"),
#                 )
# print(np.mean(perc_stat['percdiff_GFO_log']))



# #### FANCY LEGEND ################################################################
# modelnames=[]
# modelcolors = []
# modelnames.append("GRACE-FO Acc")
# modelcolors.append("black")
# modelnames.append("GRACE-FO-POD")
# modelcolors.append('#d62728')
# # modelnames.append("ICESat2 POD")
# # modelcolors.append('#1f77b4')

# df_leg = pd.DataFrame({"starts_colors": modelcolors})
# fig.update_traces(showlegend=False).add_traces(
#     [   go.Scattergl(name=modelnames[i],
#                x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
#                mode='lines',
#                line = dict(shape = 'hv',  width=10),
#                marker_color=c,
#                showlegend=True)
#         for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
# ## Legend Control
# fig.update_layout(legend=dict(
#     yanchor="top",
#     y=.78,
#     xanchor="center",
#     x=.45,
#     orientation="h",
#         font=dict(family='Arial',size=12,color='black'),
#         bgcolor="white",
#         bordercolor="darkgrey",
#         borderwidth=0.5,)  )
# ################################################################################

# # fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=3, col=1)
# # fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=4, col=1)
# # fig.add_hline(y=0, line = dict(dash='solid', color = 'grey', width=1), row=5, col=1)

# ### UPDATE AXES 
# fig.update_yaxes(title_text="Kp", 
#                  exponentformat= 'power',
#                  range=[0,7],
#                  secondary_y=False,
#                  row=1, col=1)
# fig.update_yaxes(title_text="F10.7", 
#                  exponentformat= 'power',
#                  range=[20,90],
#                  secondary_y=True,
#                  tickfont=dict(color="blue"),
#                  titlefont=dict(color="blue"),
#                  row=1, col=1)
# ################################################################################

# ### SYLIZE LEGEND 
# font_dict=dict(family='Arial',size=14,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = False
#     else:
#         if i < len(rownum):
#             L_ticklabel = False
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# # fig.update_xaxes(range=[pd.to_datetime( "181101-000000", format='%y%m%d-%H%M%S'),
# #                         pd.to_datetime( "181130-000000", format='%y%m%d-%H%M%S')],row=1, col=1)


# fig.update_yaxes(title_text="Orbit Averaged Density", 
#                  type="log", 
#                  exponentformat= 'power',row=2, col=1)
# fig.update_yaxes(title_text="Mean Ratio of Orbit Avg.", 
#                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=3, col=1,
#                     range= [0.4,1.6])
# # fig.update_yaxes(title_text="GFO_acc-ICE_pod %d", 
# #                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
# #                  exponentformat= 'power',row=4, col=1)
# # fig.update_yaxes(title_text="GFO_pod-ICE_pod %d", 
# #                 minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
# #                  exponentformat= 'power',row=5, col=1)

# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
#                   autosize=False,    width=1000,    height=800,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )

# fig.show(config=config)

# # pio.write_image(fig, 'plot_GFOacc_v_GVOpod.jpg', scale=5)


In [13]:

# gfopodcolor = "#d62728"
# icepodcolor = "#1f77b4"
# gfoacccolor = 'black'


# # fig = make_subplots(rows=1, cols=1)
# fig = make_subplots(rows=1, cols=1,#],
#                     specs=[[{"secondary_y": False}]],
#                            shared_xaxes=True,
#                            vertical_spacing=0.02)


# # fig.add_trace(go.Scattergl(x=perc_stat['ice_pod_time'],
# #                             y=perc_stat['ice_pod_denorbavg'],
# #                             name=f'D500_ice_pod',
# #                             mode='markers',
# #                             opacity=1,
# #                             marker=dict( size=4, color=icepodcolor),
# #                             showlegend=False),
# #                             row=2, col=1)



# fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_GFO'],
#                             y=perc_stat['percdiff_GFO_log'],
# #                             name=f'D500_icesat2',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=3, color=gfopodcolor),
#                             showlegend=False),
#                             row=1, col=1)
# window_size = 80
# i = 0
# # Initialize an empty list to store moving averages
# moving_averages = []
# # Loop through the array to consider
# # every window of size 3
# while i < len(perc_stat['time_midpoint_GFO']) - window_size + 1:
#     # Store elements from i to i+window_size
#     # in list to get the current window
#     window = perc_stat['percdiff_GFO_log'][i : i + window_size]
#     # Calculate the average of current window
#     window_average = round(sum(window) / window_size, 2)
#     # Store the average of current
#     # window in moving average list
#     moving_averages.append(window_average)
#     # Shift window to right by one position
#     i += 1
# fig.add_trace(go.Scattergl(x=perc_stat['time_midpoint_GFO'],
#                             y=moving_averages,
#                             opacity=1,
#                            line = dict( color =gfopodcolor, width=4),
#                             showlegend=False),
#                             row=1, col=1)







# ################################################################################

# ### SYLIZE LEGEND 
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = False
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# fig.update_yaxes(minor=dict(dtick=10, tickwidth=1,ticklen=4,tickcolor='grey',ticks='inside'),
#                  exponentformat= 'power',row=1, col=1)


# fig.update_xaxes(range=[pd.to_datetime('2019-01-14'),pd.to_datetime('2019-01-25') ],
#                  exponentformat= 'power',row=1, col=1)

# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
#                   autosize=False,    width=400,    height=100,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )

# fig.show(config=config)

# # pio.write_image(fig, 'ICEpod v GFOacc v GFOpodwgreen.jpg', scale=3)


In [14]:

# (timeavg_gfo_acc,  denavg_gfo_acc) = orbit_avg_generic(gfo_acc_df['Date'], gfo_acc_df['D500_gfo'], gfo_acc_df['lat'])    
# (timeavg_gfo_pod,  denavg_gfo_pod) = orbit_avg_generic(gfo_pod_df['date'], gfo_pod_df['D500_gfo_pod'], gfo_pod_df['lat'])    
# (timeavg_ice_pod,  denavg_ice_pod) = orbit_avg_generic(ice_pod_df['date'], ice_pod_df['D500_ice_pod'], ice_pod_df['lat'])    


# # fig = make_subplots(rows=1, cols=1)
# fig = make_subplots(rows=2, cols=1, row_heights=[0.25, 0.75],
#                     specs=[[{"secondary_y": True}],
#                            [{"secondary_y": False}]],
#                            shared_xaxes=True,
#                            vertical_spacing=0.02)

# fig.add_trace(go.Scatter(x=solar_fluxes['date'],
#                            y=solar_fluxes['f107d_earth'],
#                            name= 'F107d_1AU',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
#                            showlegend=False),
#                            secondary_y=True,row=1, col=1)

# fig.add_trace(go.Scatter(x=solar_fluxes['date_3hr'],
#                            y=solar_fluxes['kp_expand'],
#                            name= 'Kp',
#                            mode='lines',
#                            opacity=1,
#                            line = dict(shape = 'hvh', color = 'black', width=2),
#                            showlegend=False),
#                            secondary_y=False,row=1, col=1) 



# fig.add_trace(go.Scatter(x=timeavg_gfo_acc, #perc_stat['gfo_acc_time'],
#                          y=denavg_gfo_acc, #perc_stat['gfo_acc_denorbavg'],
#                         name=f'GFO_acc_500km',
#                         mode='markers',
#                         opacity=1,
#                         marker=dict( size=4, color="#d62728" ),
#                         showlegend=False),
#                         row=2, col=1)

# fig.add_trace(go.Scatter(x= timeavg_gfo_pod, #perc_stat['gfo_pod_time'],
#                          y= denavg_gfo_pod, #perc_stat['gfo_pod_denorbavg'],
#                         name=f'D500_gfo_pod',
#                         mode='markers',
#                         opacity=1,
#                         marker=dict( size=4, color="green"),
#                         showlegend=False),
#                         row=2, col=1)

# fig.add_trace(go.Scatter(x= timeavg_ice_pod, #perc_stat['ice_pod_time'],
#                          y= denavg_ice_pod, #perc_stat['ice_pod_denorbavg'],
#                         name=f'D500_ice_pod',
#                         mode='markers',
#                         opacity=1,
#                         marker=dict( size=4, color="#1f77b4"),
#                         showlegend=False),
#                         row=2, col=1)





# #### FANCY LEGEND ################################################################
# modelnames=[]
# modelcolors = []
# modelnames.append("GRACE-FO Acc")
# modelcolors.append("#d62728")
# modelnames.append("GRACE-FO POD")
# modelcolors.append('#2ca02c')
# modelnames.append("ICESat2 POD")
# modelcolors.append('#1f77b4')

# df_leg = pd.DataFrame({"starts_colors": modelcolors})
# fig.update_traces(showlegend=False).add_traces(
#     [   go.Scattergl(name=modelnames[i],
#                x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
#                mode='lines',
#                line = dict(shape = 'hv',  width=10),
#                marker_color=c,
#                showlegend=True)
#         for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
# ## Legend Control
# fig.update_layout(legend=dict(
#     yanchor="top",
#     y=.7,
#     xanchor="center",
#     x=.45,
#     orientation="h",
#         font=dict(family='Arial',size=12,color='black'),
#         bgcolor="white",
#         bordercolor="darkgrey",
#         borderwidth=0.5,)  )
# ################################################################################

# ### UPDATE AXES 
# fig.update_yaxes(title_text="Kp", 
#                  exponentformat= 'power',
#                  range=[0,7],
#                  secondary_y=False,
#                  row=1, col=1)
# fig.update_yaxes(title_text="F10.7", 
#                  exponentformat= 'power',
#                  range=[20,90],
#                  secondary_y=True,
#                  tickfont=dict(color="blue"),
#                  titlefont=dict(color="blue"),
#                  row=1, col=1)
# ################################################################################

# ### SYLIZE LEGEND 
# font_dict=dict(family='Arial',size=14,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = False
#     else:
#         if i < len(rownum):
#             L_ticklabel = False
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# fig.update_yaxes(title_text="Orbit Avg. Density [kg/m^3]", 
#                  type="log", 
#                  exponentformat= 'power',row=2, col=1)

# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
#                   autosize=False,    width=900,    height=500,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )

# fig.show(config=config)

# pio.write_image(fig, 'ICEvGFOvGFO.jpg', scale=5)


In [15]:
# small_gfo_pod_df = gfo_pod_df.query("date >= '2018-11-01' and date < '2018-11-07'")
# small_gfo_acc_df = gfo_acc_df.query("Date >= '2018-11-01' and Date < '2018-11-07'")
# small_ice_pod_df = ice_pod_df.query("date >= '2018-11-01' and date < '2018-11-07'")

# # fig = make_subplots(rows=1, cols=1)
# fig = make_subplots(rows=1, cols=1, #row_heights=[0.2, 0.4, 0.3],#,0.2],
#                            shared_xaxes=True,
#                            vertical_spacing=0.02)

# fig.add_trace(go.Scattergl(x= small_gfo_acc_df['Date'],
#                             y=small_gfo_acc_df['D500_gfo'],
#                             name=f'GFO_acc_500km',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color="#d62728" ),
#                             showlegend=False),
#                             row=1, col=1)

# fig.add_trace(go.Scattergl(x=small_gfo_pod_df['date'],
#                             y=small_gfo_pod_df['D500_gfo_pod'],
#                             name=f'D500_gfo_pod',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color="green"),
#                             showlegend=False),
#                             row=1, col=1)

# fig.add_trace(go.Scattergl(x=small_ice_pod_df['date'],
#                             y=small_ice_pod_df['D500_ice_pod'],
#                             name=f'D500_ice_pod',
#                             mode='markers',
#                             opacity=1,
#                             marker=dict( size=4, color="blue"),
#                             showlegend=False),
#                             row=1, col=1)




# #### FANCY LEGEND ################################################################
# modelnames=[]
# modelcolors = []
# modelnames.append("GraceFO Acc")
# modelcolors.append("#d62728")
# modelnames.append("GraceFO-POD")
# modelcolors.append('#2ca02c')
# df_leg = pd.DataFrame({"starts_colors": modelcolors})
# fig.update_traces(showlegend=False).add_traces(
#     [   go.Scattergl(name=modelnames[i],
#                x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
#                mode='lines',
#                line = dict(shape = 'hv',  width=10),
#                marker_color=c,
#                showlegend=True)
#         for i,c in enumerate((df_leg.loc[:,["starts_colors"]].values.ravel()))])
# ## Legend Control
# fig.update_layout(legend=dict(
#     yanchor="top",
#     y=.75,
#     xanchor="center",
#     x=.45,
#     orientation="h",
#         font=dict(family='Arial',size=12,color='black'),
#         bgcolor="white",
#         bordercolor="darkgrey",
#         borderwidth=0.5,)  )
# ################################################################################



# ### SYLIZE LEGEND 
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = False
#     else:
#         if i < len(rownum):
#             L_ticklabel = False
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=04
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)


# # fig.update_xaxes(range=[pd.to_datetime( "181101-000000", format='%y%m%d-%H%M%S'),
# #                         pd.to_datetime( "181130-000000", format='%y%m%d-%H%M%S')],row=1, col=1)


# fig.update_yaxes(title_text="Density", 
#                  type="log", 
#                  exponentformat= 'power',row=1, col=1)

# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# fig.update_layout(#title=f"ICESat2 {scale_cadence}-hr Scaled Model Rho vs GRACE-FO Rho, Norm 500km",
#                   autosize=False,    width=1000,    height=600,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict, plot_bgcolor='white', 
#                  )

# fig.show(config=config)

# # pio.write_image(fig, 'ICEvGFO.jpg', scale=5)
